These data come from a survey conducted by INSEE, Budget des Familles, which are protected by the Statistical Secrecy Committee. To reproduce the analysis, one must request access to this data.

In [9]:
import pandas as pd

In [10]:
df = pd.read_csv('BDF/Csv/C05.csv', sep=";", index_col="IDENT_MEN")

We need to remove the columns starting with 13 (taxes) and 14 (subsidies) which are not consumption. And also the columns total CTOT and pondmen (weight household) which are not useful.

In [11]:
columns_to_delete = df.columns[df.columns.str.startswith('C13') | df.columns.str.startswith('C14')]
df = df.drop(columns=columns_to_delete)
df = df.drop(columns=["CTOT", "pondmen"])

In [12]:
df_prop = df.divide(df.sum(axis=1), axis=0)


In [13]:
df_prop.rename(columns=lambda x: x.replace('C', ''), inplace=True)


We know have the proportion of each consumption area per household.

In [14]:
import eurostat

In [15]:
df_inflation_raw = eurostat.get_data_df(code="PRC_HICP_MANR", filter_pars={
                                    'geo': 'FR', 'startPeriod': "2021-03"})
df_inflation_raw = df_inflation_raw.rename(columns={'geo\TIME_PERIOD': 'geo'})

In [16]:
df_inflation = df_inflation_raw[df_inflation_raw['coicop'].str.startswith('CP') & (df_inflation_raw['coicop'].str.len() == 6)]

In [17]:
df_inflation['coicop'] = df_inflation['coicop'].str.replace('CP', '')


C:\Users\mdaversa\AppData\Local\Temp\ipykernel_13068\2914206753.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inflation['coicop'] = df_inflation['coicop'].str.replace('CP', '')


In [18]:
df_inflation = df_inflation.loc[:, ["coicop", "2023-04"]]


In [19]:
df_inflation = df_inflation.set_index('coicop').transpose()

In [20]:
for col_name in df_prop.columns:
    if str(col_name).isdigit() and len(str(col_name)) == 5:
        new_col_name = str(col_name)[:4]
        if new_col_name not in df_prop.columns:
            df_prop[new_col_name] = 0  # Initialiser la nouvelle colonne à 0
        df_prop[new_col_name] += df_prop[col_name]


C:\Users\mdaversa\AppData\Local\Temp\ipykernel_13068\1369822145.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_prop[new_col_name] = 0  # Initialiser la nouvelle colonne à 0
C:\Users\mdaversa\AppData\Local\Temp\ipykernel_13068\1369822145.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_prop[new_col_name] = 0  # Initialiser la nouvelle colonne à 0
C:\Users\mdaversa\AppData\Local\Temp\ipykernel_13068\1369822145.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

In [21]:
df_prop = df_prop.filter(regex=r'^\d{4}$')

In [22]:
missing_columns = set(df_prop.columns) - set(df_inflation.columns)
print(sorted(missing_columns))


['0131', '0221', '0241', '0311', '0331', '0412', '0450', '0461', '0521', '0541', '0571', '0631', '0641', '0714', '0741', '0811', '0812', '0813', '0814', '0943', '0961', '0971', '1011', '1012', '1013', '1014', '1015', '1113', '1121', '1233', '1241', '1251', '1255', '1261', '1271', '1281', '1291']


In [23]:
missing_columns2 = set(df_inflation.columns) - set(df_prop.columns)
print(sorted(missing_columns2))

['0442', '0612', '0613', '0820', '0830', '0935', '1213']


In [24]:
df_prop_inflation = df_prop*df_inflation